In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
from deap.tools import Statistics

# Define parameters
T = 1e-4       # Total duration of radar pulse (1e-4 seconds)
B = 1e6        # Bandwidth of radar pulse (1 MHz)
K = 5          # Number of users (OFDMA users)
L = 100        # Number of bits in radar pulse duration
Tb = T / L     # Duration of each bit interval
frequencies = np.linspace(0, B, K)  # Subcarrier frequencies linearly spaced in bandwidth B

# High-resolution time vector for plotting
t = np.linspace(0, T, 1000)

# Define LFM radar pulse
def lfm_pulse(t, T=T, B=B):
    """Linear Frequency Modulated (LFM) pulse."""
    return np.exp(1j * np.pi * B / T * t**2)

# Define OFDMA signals for each user k and bit interval l
def ofdma_signal(t, f_k, l):
    """OFDMA signal for user k in bit interval l."""
    return np.exp(1j * 2 * np.pi * f_k * (t - l * Tb)) * ((t >= l * Tb) & (t < (l + 1) * Tb))

# Construct g(t) as a matrix of OFDMA signals for all K users and L bit intervals
def construct_g_matrix():
    g_matrix = np.zeros((K, L), dtype=object)  # Storing functions instead of values
    for k in range(K):
        for l in range(L):
            g_matrix[k, l] = lambda t, k=k, l=l: ofdma_signal(t, frequencies[k], l)
    return g_matrix

# Construct the approximated radar pulse s_opt(t) using the optimal coefficients
def construct_s_opt(c_opt, t):
    s_opt = np.zeros(len(t), dtype=complex)
    for k in range(K):
        for l in range(L):
            s_opt += c_opt[k * L + l] * ofdma_signal(t, frequencies[k], l)
    return s_opt

# Normalize a signal to have unit energy
def normalize_signal(signal):
    norm = np.sqrt(np.sum(np.abs(signal)**2))
    return signal / norm

# Normalize the LFM pulse once outside the GA loop
lfm_normalized = normalize_signal(lfm_pulse(t))

# Define fitness function for GA (Mean Squared Error between LFM and approximated pulse)
def fitness(individual):
    c_opt = np.array(individual)  # Convert individual to the correct coefficients
    s_opt = construct_s_opt(c_opt, t)  # Reconstruct the approximated radar pulse
    s_opt_normalized = normalize_signal(s_opt)  # Normalize the reconstructed pulse
    mse = np.mean(np.abs(lfm_normalized - s_opt_normalized)**2)  # Calculate MSE
    return mse,

# Set up Genetic Algorithm (DEAP library)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialize population (chromosomes)
population_size = 200  # Increased population size
population = [creator.Individual(np.random.rand(K * L)) for _ in range(population_size)]  # 200 individuals

# Define crossover, mutation, and selection methods
toolbox = base.Toolbox()
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)  # Refined mutation
toolbox.register("select", tools.selTournament, tournsize=5)  # Adjusted tournament size for selection
toolbox.register("evaluate", fitness)

# Create statistics object to track and print fitness during the evolution
stats = Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)

# Run the genetic algorithm with custom statistics callback and increased generations
generations = 200  # Increased number of generations
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=generations,
                    stats=stats, halloffame=None, verbose=True)

# Extract the best solution
best_individual = tools.selBest(population, 1)[0]

# Reconstruct the approximated radar pulse with the best solution
c_opt_best = np.array(best_individual)
s_opt_best = construct_s_opt(c_opt_best, t)

# Normalize both signals to ensure energy matching
s_opt_best_normalized = normalize_signal(s_opt_best)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(t, np.real(lfm_normalized), label="Exact LFM Pulse")
plt.plot(t, np.real(s_opt_best_normalized), linestyle='--', label="Approximated Radar Pulse \(s_{opt}(t)\) using GA")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Optimal Radar Pulse Approximation using OFDMA and Genetic Algorithm (LFM Pulse)")
plt.legend()
plt.grid(True)
plt.show()